# Filter Spam 
Abraham Nieto cu 51556

In [741]:
import numpy as np 
import pandas as pd 
from sklearn.cross_validation import train_test_split 
import math
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from random import random

In [742]:
nombres = ['make','adress','all','3d','our','over','remove','internet','order','mail',
           'receive','will','people','report','addresses','free','busisnes','email','you',
           'credit','your','font','a000','money','hp','hpl','george','a650','lab','labs','telnet',
           'a857','data','a415','a85','technology','a1999','parts','pm','direct','cs','meeting',
           'original','project','re','edu','table','conference',';','(','[','!','$','#',
           'capital_run_length_average','capital_run_length_longest','capital_run_length_total','spam'
]

df=pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data',sep=',',
                header=None) 

#df=pd.DataFrame(np.array(df),columns=[nombres])

df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Hacemos data partition vams a entrenar con el **70%** de los datos

In [743]:
dftrain,dftest=train_test_split(df,test_size=0.3,random_state=101) 
dftest2=dftest.iloc[:,0:57]

vamos a calcular las medias y desviaciones estandar par cada variable en los casos que en que los mails son spam y no.

In [779]:
m0=np.mean(dftrain[dftrain[57]==0],0)[:57] #arreglo de medias de las 57 features para poblacion NOspam 
s0=np.std(dftrain[dftrain[57]==0],0)[:57] #arreglo de desv est de las 57 features para poblacion NOspam 
m1=np.mean(dftrain[dftrain[57]==1],0)[:57] #arreglo de medias de las 57 features para poblacion spam 
s1=np.std(dftrain[dftrain[57]==1],0)[:57] #arreglo de desv est de las 57 features para poblacion spam 

dado que las variables son continuas vamos a suponer que se distribuyen normal, entonces vamos a hacer la funcion de normalidad tomando en cuenta si la desv estandar es cero hay que hacer lago distinto:

$
\huge{\frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x-\mu)^{2}}{2\sigma^{2}}}}
$


In [583]:
def pnorm(x,m,s):
    if s>0.0:
        p=np.exp(-(x-m)**2/(2*s**2))*(1/np.sqrt(2*np.pi*s**2)) 
    else:
        p=1.0
    
    
    return p

Queremos clasificar si la observacion es o no spam usando Naive Bayes sabemos que...

$
P(Spam,NoSpam|x_{0},...x_{56})=P(Spam,NoSpam)*\frac{\prod_{i=0}^{56}P(x_{i}|Spam,NoSpam)}{P(x)}
$ 


Con logaritmos tenemos:

$
P(Spam,NoSpam|x_{0},...x_{56})=log(P(Spam,NoSpam))+\sum_{i=0}^{56}log(P(x_{i}|Spam,NoSpam))
$ 

y ceamos la funcion esta nos dara 2 cosas la probabilidad de pertenecer a spam y la clasificacion...

In [799]:
#programa que compara valores este se usara para la salida dela funcion de bayes
def complog(a,b):
    if a>=b:
        return 1
    else:
        return 0
    
def clasiflog(x,m0,m1,s0,s1,apriori): 
    
    spam=np.log(apriori) 
    nospam=np.log(1-apriori)
    pspam=0.0
    pnospam=0.0
    
    for i in range(len(m0)):

            
        if pnorm(x[i],m0[i],s0[i])==0.0:
            pnospam+=-len(x)*10.0
        else:    
            pnospam+=np.log(pnorm(x[i],m0[i],s0[i])) 

    for j in range(len(m1)): 

            
        if pnorm(x[j],m1[j],s1[j])==0.0: # para que no se vaya a infinito topamos el numero
            pspam+=-len(x)*10.0
        else:
            pspam+=np.log(pnorm(x[j],m1[j],s1[j])) 
      

    pspam+=spam
    pnospam+=nospam
    
    
    if np.exp(pspam)+np.exp(pnospam)!=0:
        return (np.exp(pspam)/(np.exp(pspam)+np.exp(pnospam))),complog(pspam,pnospam)
    else:
        return 0,0

In [800]:
clasiflog(dftest2.iloc[40],m0,m1,s0,s1,ap)

(3.0569995706712229e-235, 0)

Corremos la prediccion para cada observacion y tenemos un arreglo de predicciones 'resx'

In [801]:
%%time
resx=np.zeros((len(dftest2),2))
for i in range(len(dftest2)):
    resx[i]=clasiflog(dftest2.iloc[i],m0,m1,s0,s1,ap)
resx  

CPU times: user 10 s, sys: 4 ms, total: 10 s
Wall time: 10 s


**Matriz de confusion de nuestra funcion clasiflog**

In [802]:
confusion_matrix(dftest[57],resx[:,1])

array([[592, 232],
       [ 27, 530]])

# Comprobacion con sklearn

In [798]:
X_train=np.array(dftrain.iloc[:,0:57])
y_train=np.array(dftrain.iloc[:,57]) 
X_test=np.array(dftest.iloc[:,0:57])
y_test=np.array(dftest.iloc[:,57])

from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(X_train,y_train) 
pred=NB.predict(X_test) 
print(confusion_matrix(y_test,pred))

[[595 229]
 [ 23 534]]
